# **Import Data Library**

In [10]:
import pandas as pd                         # Import Pandas
import statsmodels.api as sm                                  
data = pd.read_csv("cleanDiamods.csv")      # Import dataset 

# **Split Train and Test**

In [11]:
train = data.head(int(len(data)*0.7))
test = data.tail(int(len(data)*0.3))

In [12]:
y = train["price"]
x = train[["carat","x","y","z","table","depth","cut","color","clarity"]]
x = sm.add_constant(x)

In [13]:
mod = sm.OLS(y, x)                          # Create Modal
results = mod.fit()                         # Fit Modal
results.summary()                 

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.907
Model:                            OLS   Adj. R-squared:                  0.907
Method:                 Least Squares   F-statistic:                 4.078e+04
Date:                Thu, 11 Nov 2021   Prob (F-statistic):               0.00
Time:                        20:06:55   Log-Likelihood:            -3.2549e+05
No. Observations:               37758   AIC:                         6.510e+05
Df Residuals:                   37748   BIC:                         6.511e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.039e+04    588.064     17.664      0.000    9235.031    1.15e+04
carat       1.049e+04     62.604    167.531      0.000    1.04e+04    1.06e+04
x           -665.8504     55.213    -12.060      0.000    -774.070    -557.631
y             50.5797     26.248      1.927      0.054      -0.868     102.027
z            -80.3231     78.185     -1.027      0.304    -233.567      72.921
table        -31.3266      3.900     -8.032      0.000     -38.971     -23.682
depth        -76.8191      7.440    -10.325      0.000     -91.402     -62.236
cut         -155.6869      7.583    -20.532      0.000    -170.549    -140.825
color       -402.1996      4.303    -93.469      0.000    -410.634    -393.766
clarity     -637.7539      4.625   -137.882      0.000    -646.820    -628.688
==============================================================================
Omnibus:                     7061.194   Durbin-Watson:                   1.102
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           191499.274
Skew:                          -0.148   Prob(JB):                         0.00
Kurtosis:                      14.029   Cond. No.                     7.28e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.28e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [14]:
price_predect = []
for i in range(test.index[0], test.index[-1] + 1):
    price_predect.append(+1.039e+04
                    +1.049e+04 *test["carat"][i] 
                    -665.8504 *test["x"][i]
                    +50.5797 *test["y"][i] 
                    -80.3231 *test["z"][i]
                    -31.3266 *test["table"][i] 
                    -76.8191 *test["depth"][i]
                    -155.6869 *test["cut"][i]
                    -402.1996 *test["color"][i]
                    -637.7539 *test["clarity"][i])

indexs = [i for i in range(test.index[-1] + 1 - test.index[0])]
d = dict(zip(indexs,price_predect) )


In [15]:
predict_data = pd.concat([test.reset_index() , pd.Series(d, name='predict')], axis = 'columns')
predict_data

,index,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z,predict
0,37758,37758,0.38,0,3,4,62.5,56.0,998,4.64,4.58,2.88,973.880892
1,37759,37759,0.44,1,1,6,62.3,58.0,998,4.88,4.85,3.03,770.999970
2,37760,37760,0.34,2,2,1,61.1,54.0,999,4.54,4.56,2.78,2802.174088
3,37761,37761,0.53,0,6,6,61.6,55.0,999,5.20,5.23,3.21,-200.767960
4,37762,37762,0.34,0,2,1,61.8,55.0,999,4.49,4.51,2.78,3059.211453
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16177,53935,53935,0.72,0,0,6,60.8,57.0,2757,5.75,5.76,3.50,3841.627542
16178,53936,53936,0.72,3,0,6,63.1,55.0,2757,5.69,5.75,3.61,3291.145798
16179,53937,53937,0.70,2,0,6,62.8,60.0,2757,5.66,5.68,3.56,3123.896516
16180,53938,53938,0.86,1,4,7,61.0,58.0,2757,6.15,6.12,3.74,2593.888910


In [16]:
ans = []
for i in predict_data.index:
    d = predict_data["price"][i] - predict_data["predict"][i]
    ans.append(abs(d))
print("abs meen error =",sum(ans) / len(ans))

abs meen error = 745.6221653239418


In [17]:
import math
summm=0
for i in predict_data.index:
    d = predict_data["price"][i] - predict_data["predict"][i]
    summm+=d**2

rmsee=math.sqrt(summm/len(predict_data))
print("rmsee =",rmsee)

rmsee = 928.4877965457805


In [18]:
summm=0
for i in predict_data.index:
    d=predict_data["price"][i]-predict_data["predict"][i]
    summm+=abs(d)/predict_data["price"][i]

mape=(summm/len(predict_data))*100
print("mape =",mape)

mape = 62.81872364228498
